In [2]:
!pip install -U --quiet duckduckgo_search==5.3.1b1
!pip install -U --quiet ibm-watsonx-ai


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
# Utils
import re
import tempfile
from dotenv import load_dotenv
import os
import json
import re
from typing import List
import time
from joblib import Parallel, delayed

# Watson Discovery
from ibm_watson import DiscoveryV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

# Watsonx
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from ibm_watsonx_ai import Credentials
from langchain_ibm import WatsonxLLM

# Langchain

from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_community.tools import DuckDuckGoSearchResults
from langchain.schema import Document

In [4]:

load_dotenv()

credentials = Credentials(
                   url = os.getenv('WATSONX_URL'),
                   api_key = os.getenv('WATSONX_APIKEY'),
                  )
try:
    project_id = os.getenv("WATSONX_INSTANCE_ID")
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")
    

authenticator = IAMAuthenticator(os.getenv('WATSONX_APIKEY'))
discovery_project_id = os.getenv("DISCOVERY_PROJECT_ID")

In [5]:

model_id = "meta-llama/llama-3-70b-instruct"

# Greedy model for most prompts
parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 200,
}


watsonx_llama3 = WatsonxLLM(
    model_id=model_id,
    url=credentials.get("url"),
    apikey=credentials.get("apikey"),
    project_id=project_id,
    params=parameters
)


In [6]:
## test if the model is reasonable:
watsonx_llama3.invoke('Argue if capitalism is good or bad for society. Summarize your conclusions in a concise sentences.')

' Capitalism is a complex and multifaceted economic system that has both positive and negative impacts on society. While it has lifted millions of people out of poverty, a significant portion of the population remains in poverty, and income inequality has increased. Capitalism promotes innovation, efficiency, and economic growth, but it also creates social and environmental problems, such as exploitation of workers, environmental degradation, and concentration of wealth. Ultimately, whether capitalism is good or bad for society depends on how it is regulated and implemented, and whether its benefits are shared equitably among all members of society.'

In [7]:
### Search - DuckDuckGo

web_search_tool = DuckDuckGoSearchResults(max_results=5)

docs = web_search_tool.invoke("CEO of IBM?")

def parse_ddg_results(input_string:str) -> List[str]:
    # Regex to match each entry in the string
    pattern = r'\[snippet: (.*?), title: (.*?), link: (.*?)\]'
    # Find all matches using regex
    matches = re.findall(pattern, input_string)
    # Create a list of dictionaries based on the matches
    data = [{'snippet': match[0], 'title': match[1], 'link': match[2]} for match in matches]
    return data

# Output the result
print(parse_ddg_results(docs))


[{'snippet': "Krishna talked with CNBC about his specific views on regulation, the business of generative AI and IBM's successes and mistakes. IBM CEO Arvind Krishna speaks at a panel session at the World ...", 'title': 'IBM CEO Arvind Krishna CNBC interview', 'link': 'https://www.cnbc.com/2023/12/07/ibm-ceo-arvind-krishna-cnbc-interview.html'}, {'snippet': 'Lou Gerstner (born March 1, 1942, Mineola, New York, U.S.) is an American businessman best known for the pivotal role he played in revitalizing the ailing IBM in the mid-1990s; he served as CEO of the company from 1993 to 2002. Gerstner studied engineering at Dartmouth College in Hanover, New Hampshire (B.A., 1963), where he graduated magna cum ...', 'title': 'Lou Gerstner | Biography, IBM, & Facts | Britannica Money', 'link': 'https://www.britannica.com/money/Lou-Gerstner'}, {'snippet': 'IBM did see momentum in its AI business, however. The company revealed it had more than $1 billion in AI-related backlog. Krishna says companies 

## Discovery Related Prompts

In [8]:
summarize_collection_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are helpful assistant writing a short summary so a user know's what's included. 
    Summarize the folowing retrieved context. Limit your summary to 3 sentences. Don't include extraneous information! 
    Only respond with the summary with no preamble or explanation.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Context: {context} 
    Summary: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["context"],
)

summary_chain = summarize_collection_prompt | watsonx_llama3 | StrOutputParser()

name_collection_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are helpful assistant writing a topic name to assign this question. 
    Please provide a helpful topic name for this question and similar questions. Generally, it can be an IBM product name.
    Only respond with the topic name with no preamble or explanation.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question} 
    Topic Name: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question"],
)

name_collection_chain = name_collection_prompt | watsonx_llama3 | StrOutputParser()

## Discovery Wrapper Functions

In [9]:
# Watson Discovery Utils

def clean_text(text) -> str:
    """description"""
    # Remove HTML tags
    clean = re.compile('<.*?>')
    cleaned_text = re.sub(clean, '', text)
    
    # Replace multiple newline characters with a single newline
    cleaned_text = re.sub(r'\n+', '\n', cleaned_text)
    
    # Remove leading and trailing whitespace
    cleaned_text = cleaned_text.strip()
    
    return cleaned_text

def watson_discovery_tool(query: str) -> str:
    """
    A tool to query IBM Watson Discovery for insights from unstructured data.
    The input query should be a natural language query.
    """
    discovery = DiscoveryV2(version="2019-11-29", authenticator=authenticator)
    discovery.set_service_url(os.getenv("DISCOVERY_SERVICE_URL"))

    response = discovery.query(project_id=discovery_project_id, natural_language_query=query).get_result()
    results = response["results"]
    passages = [x["document_passages"] for x in results]
    excerpts = [x["passage_text"]  for l in passages for x in l]
    context = "\n\n".join(excerpts)

    return clean_text(context)

def watson_discovery_summary_collections() -> str:
    """
    A tool to query IBM Watson Discovery for insights from unstructured data.
    The input query should be a natural language query.
    """

    discovery = DiscoveryV2(version="2019-11-29", authenticator=authenticator)
    discovery.set_service_url(os.getenv("DISCOVERY_SERVICE_URL"))

    # List collections
    collections = discovery.list_collections(project_id=discovery_project_id).get_result()
    # Extract collection IDs
    collection_ids = [collection['collection_id'] for collection in collections['collections']]
    # Initialize an empty dictionary
    collection_summaries = {}


    for collection in collection_ids:
        response = discovery.query(project_id="ab6d11d1-e16e-4b87-9efe-3d1c5f2ce17a", collection_ids= [collection], natural_language_query="").get_result()
        results = response["results"]
        passages = [x["document_passages"] for x in results]
        excerpts = [x["passage_text"]  for l in passages for x in l]
        context = "\n\n".join(excerpts)

        # Run chain
        context = clean_text(context)
        summary = summary_chain.invoke({"context": context})
        collection_summaries[collection] = clean_text(summary)


    return collection_summaries

def watson_discovery_add_document(question: str, answer: str, collection_id) -> None:
    """
    A function to add a document to a specific collection in IBM Watson Discovery.
    """
    discovery = DiscoveryV2(version="2019-11-29", authenticator=authenticator)
    discovery.set_service_url(os.getenv("DISCOVERY_SERVICE_URL"))

    question = question.strip()
    answer = answer.strip()
    document_content = f"USER QUESTION: {question}\n\nUSEFUL ANSWER:\n{answer}"
    print(document_content)
    print("------------")
    with tempfile.NamedTemporaryFile(mode='w', delete=False) as temp_file:
            temp_file.write(document_content)
            temp_file_path = temp_file.name

    # Add the document to Watson Discovery using the temporary file
    with open(temp_file_path, 'rb') as file:
        response = discovery.add_document(
            project_id=discovery_project_id,
            collection_id=collection_id,
            file=file
        ).get_result()


    os.remove(temp_file_path)
    
    print(f"Document added successfully. Document ID: {response['document_id']}")
    
def watson_discovery_create_collection(question: str) -> str:
    """
    A function to create a collection in IBM Watson Discovery.
    """
    # Chain
    
    name = name_collection_chain.invoke({"question": question}).strip()
    discovery = DiscoveryV2(version="2019-11-29", authenticator=authenticator)
    discovery.set_service_url(os.getenv("DISCOVERY_SERVICE_URL"))

    response = discovery.create_collection(project_id=discovery_project_id,
                                name=name).get_result()
    
    
    print(f"New Collection successfully created. \n Name: {name} \n Collection ID: {response['collection_id']}")
    
    return response['collection_id']

In [10]:
watson_discovery_tool("what foundation models are available?")

"with model training, development, visual modeling, and synthetic data generation IBM watsonx.ai Foundation Models Library – available today IBM Granite Llama 3 models LAB Aligned models © © A © © © © granite\nOther product and service names might be trademarks of IBM or other companies. A current list of IBM trademarks is available on ibm.com/trademark. Foundation models and Generative AI are bringing an inflection point in AI\n| | \\ > F | ( S / > C | ¢ | | | | % 2 © 2023 IBM Corporation IBM's AI assistants use Foundation Models and Automation to orchestrate skill execution and reduce time and effort Customers, employees, and\nSource links: Foundation model parameters: decoding and stopping criteria Foundation models Choosing a foundation model in watsonx. ai © © ® 14 © 2023 IBM Corporation Advantages of Retrieval Augmented Generation (RAG) RAG is a form of architecture that uses search results to enhance the generative model.\n. | & | • Create a custom language model simply by uploa

In [11]:
collection_summaries = watson_discovery_summary_collections()
print(collection_summaries)

{'e7b5cf24-2148-7ed5-0000-018f54532efc': 'IBM Watsonx is an AI and data platform that helps deploy and embed AI across businesses, with high-impact use cases like chatbots. Watson Assistant is a conversational AI application that can be used to quickly deploy chatbots. In Watson Assistant, you can use foundation models like IBM Granite Code LLM and Code Llama.', '6042175d-e2c0-b031-0000-018f5451ff1d': 'Here is a 3-sentence summary:\nIBM Watsonx.data is an open, hybrid, and governed data store that helps scale AI workloads by providing a trusted data foundation. It offers fit-for-purpose query engines, built-in data governance, security, and automation. A data fabric architecture can simplify data access for users, addressing the challenges of dispersed, dynamic, and diverse data.', '583b032f-e9f6-8596-0000-018f5452b6b3': "Watsonx Orchestrate is an intelligent assistant that boosts productivity by efficiently resolving tasks across platforms. It's part of IBM's strategy to implement AI 

In [12]:
# watson_discovery_create_collection("what is IBM Knowledge Catalog?")

## Helper Prompts for QA Pipeline

In [13]:


retrieval_grader_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing relevance 
    of a retrieved document to a user question. If the document contains any information useful to answer user's question, 
    grade it as relevant. If the document contains information that is relevant to only a portion of the question, grade it as relevant.
    The goal is to filter out completely irrelevant retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explaination.
     <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question", "document"],
)

retrieval_grader = retrieval_grader_prompt | watsonx_llama3 | JsonOutputParser()
question = "What are Watson Orchestrate skills?"
docs = watson_discovery_tool(question)
print(retrieval_grader.invoke({"question": question, "document": docs}))

{'score': 'yes'}


The above should be yes -- needs fixing.

In [14]:

# Prompt
reword_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an in charge of querying a search engine on behalf of a user.
    A user will provide you a question that you need to reword for optimal search on business documents. If there are multiple questions, break them down into multiple queries.
    For example, if a user asks to compare the forecast in Chicago to New York, you should form queries such as "Weather forecast NYC", "Weather forecast Chicago", "current weather NYC", "current weather Chicago", etc.
    Provide up to 4 queries in a JSON list with a single key 'query' for each query and no premable or explaination.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question"],
)

# Chain
reword_chain = reword_prompt | watsonx_llama3 | JsonOutputParser()

# Run
question = "What are Watson Orchestrate skills?"
generation = reword_chain.invoke({"question": question})
print(generation)

[{'query': 'Watson Orchestrate skills list'}, {'query': 'Watson Orchestrate features'}, {'query': 'IBM Watson Orchestrate capabilities'}, {'query': 'Watson Orchestrate AI skills'}]


In [15]:

# Prompt
reword_feedback_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an in charge of querying a search engine on behalf of a user.
    Analyze the current question and corresponding document. The current query is not optimal and needs to be reworded for better results. 
    Provide the query as a JSON with a single key 'query' and no preamble or explanation.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Query: {query} 
    Document: {document} 
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["query", "document"],
)

# Chain
reword_feedback_chain = reword_feedback_prompt | watsonx_llama3 | JsonOutputParser()

# Run

question = "What are Watson Orchestrate skills?"
docs = watson_discovery_tool(question)
print(reword_feedback_chain.invoke({"query": question, "document": docs}))

{'query': 'What are the skills of Watson Orchestrate Product Manager?'}


In [16]:

# Prompt
rag_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. 
    Answer the question completely while keeping the answer concise. Don't include extraneous information <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question} 
    Context: {context} 
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "context"],
)


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = rag_prompt | watsonx_llama3 | StrOutputParser()

# Run
question = "What are Watson Orchestrate skills?"
docs = watson_discovery_tool(question)
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)



Watson Orchestrate skills are AI-powered skills that leverage Gen AI to accomplish new tasks or build new skills, and execute automations. They are part of the IBM Watson Orchestrate platform, which helps companies empower and scale their workforce by democratizing automations through digital workers and assistants.


In [17]:
### Hallucination Grader 

# Prompt
hallucination_grader_prompt = PromptTemplate(
    template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether 
    an answer is grounded in / supported by a set of facts. Give a binary score 'yes' or 'no' score to indicate 
    whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a 
    single key 'score' and no preamble or explanation. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here are the facts:
    \n ------- \n
    {documents} 
    \n ------- \n
    Here is the answer: {generation}  <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "documents"],
)

hallucination_grader = hallucination_grader_prompt | watsonx_llama3 | JsonOutputParser()
hallucination_grader.invoke({"documents": docs, "generation": generation})

{'score': 'no'}

In [18]:
### Answer Grader 

# Prompt
answer_grader_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether an 
    answer is useful to resolve a question. Give a binary score 'yes' or 'no' to indicate whether the answer is 
    useful to resolve a question. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
     <|eot_id|><|start_header_id|>user<|end_header_id|> Here is the answer:
    \n ------- \n
    {generation} 
    \n ------- \n
    Here is the question: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "question"],
)

answer_grader = answer_grader_prompt | watsonx_llama3 | JsonOutputParser()
answer_grader.invoke({"question": question,"generation": generation})

{'score': 'yes'}

In [19]:
def construct_query_router_prompt(collection_summaries):

    template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|> 
    You are an expert at routing a query to the appropriate data source.
    Use the following criteria to determine the data source:\n"""
    for cid, summary in collection_summaries.items():
        template = template + f"\t- {cid}: {summary}\n"

    template = template + """\t- web_search: If the query does not match any of the collections above.

    Provide your decision as a JSON object with a single key 'datasource' and no preamble or explanation.
    Query to route: {query} 
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

    route_query_prompt = PromptTemplate(
    template=template,
    input_variables=["query"],
    )
    
    # print(template)

    # Chain
    # route_query_chain = route_query_prompt | watsonx_llama3 | JsonOutputParser()
    
    return route_query_prompt

In [20]:
route_query_prompt = construct_query_router_prompt(collection_summaries)
route_query_chain = route_query_prompt | watsonx_llama3 | JsonOutputParser()

query = "What are Watson Orchestrate skills?"
route_query_chain.invoke({"query": query})

{'datasource': '583b032f-e9f6-8596-0000-018f5452b6b3'}

## Implement chains above as a control flow.

In [21]:

def generate(state):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    
    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}

def grade_document(query, document, datasource):
    """
    Determines whether the retrieved document are relevant to the query
    If the document is not relevant, we will run web search

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Filtered out irrelevant documents and updated web_search state
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")

    # Score each doc
    filtered_docs = []
    #for query, document, datasource in zip(state['queries'], documents, datasources):
    result = retrieval_grader.invoke({"question": query, "document": document.page_content})
    grade = result['score']
    # Document relevant
    if grade.lower() == "yes":
        print("---GRADE: DOCUMENT RELEVANT---")
        return document
    # Document not relevant
    else:
        print("---GRADE: DOCUMENT NOT RELEVANT---")
        # Invoke web search to replace irrelevant document
        if datasource != "web_search":
            document = web_search(query)
            result = retrieval_grader.invoke({"question": query, "document": document.page_content})
            grade = result['score']
       # Generate new web results to replace irrelevant web search
        i = 0
        while grade.lower() != "yes" and i < 3:
            print("---REWORDING WEB QUERY---")

            query = reword_feedback_chain.invoke({"query": query, "document": document.page_content})
            document = web_search(query['query'])
            result = retrieval_grader.invoke({"question": query, "document": document.page_content})
            grade = result['score']

            i+= 1
        return document

    
    
def web_search(query):
    """
    Web search based based on the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web results to documents
    """

    print("---WEB SEARCH---")
    # Web search
    str_docs = web_search_tool.invoke({"query": query})
    docs = parse_ddg_results(str_docs)
    web_results = "\n".join([d["snippet"] for d in docs])
    web_results = Document(page_content=web_results)
    
    documents = web_results
    
    return documents

In [22]:

def query_analyzer(state):
    """
    Break down the question into multiple queries for search.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updated state with generated queries
    """
    print("---QUERY ANALYZER---")
    question = state["question"]
    
    # Generate queries using the reword_chain
    queries = reword_chain.invoke({"question": question})
    
    # Append original question as a query as well
    queries.append({"query": question})
    
    return {"question": question, "queries": queries}

def route_query(query):
    """
    Route query to Watson Discovery or web search using an LLM.

    Args:
        state (dict): The current graph state
        query (str): The query to route

    Returns:
        str: Next node to call
    """
    print("---ROUTE QUERY---")
    
    # Use the LLM to determine the data source
    result = route_query_chain.invoke({"query": query})
    datasource = result['datasource']
    
    return datasource



def watson_discovery_retrieve(query, collection):
    """
    Retrieve documents from Watson Discovery based on the collection.

    Args:
        state (dict): The current graph state
        collection (str): The selected collection

    Returns:
        state (dict): Updated state with retrieved documents
    """
    print(f"---WATSON DISCOVERY RETRIEVE ({collection})---")
    
    # Retrieve documents from Watson Discovery
    result = watson_discovery_tool(query)
    
    disc_results = Document(page_content=result)
    
    documents = disc_results
    
    return documents


    
def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke({"documents": documents, "generation": generation})
    grade = score['score']

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question,"generation": generation})
        grade = score['score']
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"


In [23]:
def process_query(user_question, query, datasource):
    print(f"Processing query: {query}")
    
    # Route the query
    print("Datasource: " + datasource)
    
    if datasource == "web_search":
        documents = web_search(query)
    else:
        try:
            summary = collection_summaries[datasource]
        except:
            print("Router returned a collection ID not in dictionary")
            return []
        documents = watson_discovery_retrieve(query, datasource)
        
    
    documents = grade_document(query, documents, datasource)
    return documents

def run_pipeline(question):
    # Get the initial question
    state = {"question": question}
    
    # Run the query analyzer
    state = query_analyzer(state)
    queries = state["queries"]

    state["datasources"] = Parallel(n_jobs=-1, prefer="threads")(delayed(route_query)(query['query']) for query in queries)
    datasources = state["datasources"]
    
    # Process each query in parallel using joblib
    documents = Parallel(n_jobs=-1, prefer="threads")(delayed(process_query)(state["question"], query['query'], datasource)
                                                      for query, datasource in zip(queries, datasources))
    
    state["documents"] = documents
    
    #state = grade_documents(state)
    state = generate(state)
    state["answer_class"] = grade_generation_v_documents_and_question(state)
    # Print the final results
    print("Final Answer:\n" + state["generation"])
    
    return state

In [29]:

start = time.time()
# question = "How does watson orchestrate differ from watson assistant?"
# question = "How does IBM Knowledge Catalog compare to Informatica?"
question = "What foundation models can I use in watson assistant and how do they compare performatively to OpenAI?"
result = run_pipeline(question)

end = time.time()
print("Time to execute: " + str(end - start))

---QUERY ANALYZER---
---ROUTE QUERY---
---ROUTE QUERY---
---ROUTE QUERY---
---ROUTE QUERY---
---ROUTE QUERY---
Processing query: Watson Assistant foundation modelsProcessing query: Watson Assistant AI models comparison
Datasource: e7b5cf24-2148-7ed5-0000-018f54532efc
---WATSON DISCOVERY RETRIEVE (e7b5cf24-2148-7ed5-0000-018f54532efc)---

Datasource: e7b5cf24-2148-7ed5-0000-018f54532efc
---WATSON DISCOVERY RETRIEVE (e7b5cf24-2148-7ed5-0000-018f54532efc)---
Processing query: OpenAI models vs Watson Assistant performance
Datasource: e7b5cf24-2148-7ed5-0000-018f54532efc
---WATSON DISCOVERY RETRIEVE (e7b5cf24-2148-7ed5-0000-018f54532efc)---
Processing query: Foundation models in Watson Assistant vs OpenAI
Datasource: e7b5cf24-2148-7ed5-0000-018f54532efc
---WATSON DISCOVERY RETRIEVE (e7b5cf24-2148-7ed5-0000-018f54532efc)---
Processing query: What foundation models can I use in watson assistant and how do they compare performatively to OpenAI?
Datasource: e7b5cf24-2148-7ed5-0000-018f54532efc


In [27]:
if result["answer_class"] == "useful":
    collection_id = route_query_chain.invoke({"query": question})["datasource"]
    
    if collection_id == "web_search": # There is no relevant collection so let's create one
        collection_id = watson_discovery_create_collection(question)
        summary = summary_chain.invoke({"context": result["generation"]})
        collection_summaries[collection_id] = summary
        
    watson_discovery_add_document(question, result["generation"], collection_id)

New Collection successfully created. 
 Name: IBM Cloud Pak for Data 
 Collection ID: d34c4f1e-dd5b-2737-0000-018f976cd037
USER QUESTION: How does IBM Knowledge Catalog compare to Informatica?

USEFUL ANSWER:
IBM Knowledge Catalog and Informatica Enterprise Data Catalog are both data cataloging solutions that provide features for data governance, data quality, and data discovery. They share some similarities, such as offering data lineage tracking and data quality scores. However, they also have some differences, such as IBM Knowledge Catalog's focus on AI-assisted self-service discovery and its integration with Cloud Pak for Data as a Service, whereas Informatica Enterprise Data Catalog is powered by a machine-learning-based discovery engine.
------------
Document added successfully. Document ID: f3864e65-a3ca-4797-9a24-8cbd99e3e6bb
